In [1]:
import tkinter as tk
from tkinter import messagebox
import time
import random
import math

In [2]:
class CubicGame():
    
    def __init__(self): #Initialize board and set currentWinner to 'None' 
        self.board = self.makeBoard()
        self.currentWinner = None
        
    def makeBoard(self): #Returning a board of length 16 with 'Spaces'
        return [' ' for _ in range(16)]
    
    def printBoard(self): #Going through the four rows and print '| | |'
        for row in [self.board[i * 4: (i + 1) * 4] for i in range(4)]:
            print('| ' + ' | '.join(row) + ' |')
            
    def printBoardNumbers(self): #Assigns a value 0 through 15 to every single space on the board
        # 0 | 1 | 2 | 3 ... and so on
        numBoard = [[str(i) for i in range(j * 4, (j + 1) * 4)] for j in range(4)]
        
        for row in numBoard:
            print('| ' + ' | '.join(row) + ' |')
                    
    
    def makeMove(self, square, letter): #Move on the board
        #square representing which space the user want to go, number between 0 and 15
        #letter representing which is 'X' or 'O'
        if self.board[square] == ' ': #If space empty
            self.board[square] = letter #Assign the letter to that square
            if self.winner(square, letter): #If it's the winner
                self.currentWinner = letter #Set currentWinner to the current player 'letter X or O'
            return True
        return False
    
    def winner(self, square, letter): #To checking if the move that i made, made me a winner or not
        rowInd = math.floor(square / 4)
        row = self.board[rowInd * 4: (rowInd + 1) * 4] #Getting the row in the board
        if all([single == letter for single in row]): #If every single letter in column is the same letter
            return True
        
        colInd = square % 4
        column = [self.board[colInd + i * 4] for i in range(4)] #Getting the column in the board
        if all([single == letter for single in column]): #If every single letter in column is the same letter
            return True  
                  
        if square % 3 == 0 and square % 15 != 0:
            diagonal_1 = [self.board[i] for i in [0, 5, 10, 15]]
            if all([single == letter for single in diagonal_1]): #If every single letter in diagonal_1 is the same letter
                      return True 
                  
            diagonal_2 = [self.board[i] for i in [3, 6, 9, 12]]
            if all([single == letter for single in diagonal_2]): #If every single letter in diagonal_2 is the same letter
                      return True
        return False
    
    def emptySquares(self): #To check if there are more empty squraes on the board
        return ' ' in self.board
                  
    def numEmptySquares(self): #Count number of empty squares on the board
        return self.board.count(' ')
                  
    def availableMoves(self): #Getting numerical value of the spaces that are still empty
        return [i for i, x in enumerate(self.board) if x == " "]
        
    def evaluateHeuristic(self):
        #Check rows for winning patterns
        for i in range(4):
            row = self.board[i * 4: (i + 1) * 4]
            if all(cell == 'X' for cell in row):
                return 10  #Player X wins
            elif all(cell == 'O' for cell in row):
                return -10 #Player O wins
        #Check columns for winning patterns
        for i in range(4):
            column = [self.board[j * 4 + i] for j in range(4)]
            if all(cell == 'X' for cell in column):
                return 10  # Player X wins
            elif all(cell == 'O' for cell in column):
                return -10  # Player O wins
        
        #Check diagonals for winning patterns
        diagonal_1 = [self.board[i] for i in [0, 5, 10, 15]]
        if all(cell == 'X' for cell in diagonal_1):
            return 10  #Player X wins
        elif all(cell == 'O' for cell in diagonal_1):
            return -10  #Player O wins

        diagonal_2 = [self.board[i] for i in [3, 6, 9, 12]]
        if all(cell == 'X' for cell in diagonal_2):
            return 10  #Player X wins
        elif all(cell == 'O' for cell in diagonal_2):
            return -10  #Player O wins
        
        #No winner yet so will return '0' for tie or 'None' if the game is ongoing
        if not self.emptySquares():
            return 0  #Tie
        else:
            return None  #Game ongoing

In [3]:
class Player(): #Super class called player
    
    def __init__(self, letter):
        self.letter = letter
        
    def getMove(self, game):
        pass

In [4]:
class Ai_Player(Player):
    
    def __init__(self, letter):
        super().__init__(letter)
        
    def getMove(self, game):
        if len(game.availableMoves()) == 16: #If it's an empty game
            square = random.choice(game.availableMoves()) #Randomly choose one of those squares to go 
        else:
            square = self.minimax(game, self.letter, -math.inf, math.inf, True)['Position'] #Use Minimax Algorithm, to generate smartest possible choice
        return square
    
    def minimax(self, state, player, alpha, beta, maximizingPlayer, depth=0, maxDepth=3):
        
        #state representing the current state of the game
        #player representing the player that going on this next turn
        
        maxPlayer = self.letter
        otherPlayer = 'O' if player == 'X' else 'X'
        
        #Check if the previous move is winner or if max depth is reached
        if depth == maxDepth or state.currentWinner:
            return {'Position': None, 'Score': 1 * (state.numEmptySquares() + 1) if state.currentWinner == self.letter
                    else -1 * (state.numEmptySquares() + 1)}

        elif not state.emptySquares(): #If no more squares left
            return {'Position': None, 'Score': 0}

        if maximizingPlayer:
            best = {'Position': None, 'Score': -math.inf} #Each score should maximize
            for possibleMove in state.availableMoves():
                state.makeMove(possibleMove, player)
                simScore = self.minimax(state, otherPlayer, alpha, beta, False, depth + 1, maxDepth) 
                state.board[possibleMove] = ' '
                state.currentWinner = None
                simScore['Position'] = possibleMove #Represents the move optimal next move
                
                best = max(best, simScore, key=lambda x: x['Score'])
                alpha = max(alpha, best['Score'])
                if beta <= alpha:
                    break
            return best
        
        else:
            best = {'Position': None, 'Score': math.inf} #Each score should minimize
            for possibleMove in state.availableMoves():
                state.makeMove(possibleMove, player)
                simScore = self.minimax(state, otherPlayer, alpha, beta, True, depth + 1, maxDepth) 
                state.board[possibleMove] = ' '
                state.currentWinner = None
                simScore['Position'] = possibleMove  #Represents the move optimal next move
                
                best = min(best, simScore, key=lambda x: x['Score'])
                beta = min(beta, best['Score'])
                if beta <= alpha:
                    break
            return best

In [5]:
class CubicTicTacToe4x4x4GUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Cubic Tic Tac Toe 4x4x4") #Tittle of the window

        self.game = CubicGame() #Object of CubicGame Class to use our logic

        self.buttons = [] #Buttons for board
        for i in range(4):
            for j in range(4):
                button = tk.Button(root, text=' ', font=('Arial', 20), width=6, height=3,
                                   command=lambda i=i, j=j: self.makeMove(i, j)) #Button for each cell in the board, command call makeMove function inside lambda function
                button.grid(row=i, column=j)
                self.buttons.append(button) #add button to list
        #Initialize players
        self.Human_player = 'O'
        self.Ai_player = Ai_Player('X')

    def makeMove(self, row, col):
        index = row * 4 + col #calc. the index of selected cell
        if self.game.makeMove(index, self.Human_player):
            self.buttons[index].config(text=self.Human_player) #to show in the index human player symbol
            #check if is winner or if it's tie
            if self.game.currentWinner:
                messagebox.showinfo("Winner", f"Player {self.game.currentWinner} wins !!") #Winner Appears on title of tab and wins its the message
                self.resetGame()
            elif not self.game.emptySquares():
                messagebox.showinfo("Tie", "It's a tie !!") #Tie Appears on title of tab and It's tie its the message
                self.resetGame()
            else:
                self.Human_player = 'O' if self.Human_player == 'X' else 'X'
                if self.Human_player == 'X':
                    self.makeAiMove()

    def makeAiMove(self): #Make Ai move and get the move from its class
        move = self.Ai_player.getMove(self.game)
        self.makeMove(move // 4, move % 4) #to move on the board

    def resetGame(self): #Reset game
        self.game = CubicGame()
        for button in self.buttons: #loop on the buttons list
            button.config(text=' ') #make all buttons empty
        self.Human_player = 'O'

In [6]:
if __name__ == "__main__":
    root = tk.Tk()
    CallingGUI = CubicTicTacToe4x4x4GUI(root)
    root.mainloop()